In [1]:
import requests
import os
import json
import pandas as pd 
import folium
import spacy
import geopy 
# import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter
from folium.plugins import FastMarkerCluster

locator = geopy.geocoders.Nominatim(user_agent="mygeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=5)
nlp_wk = spacy.load('xx_ent_wiki_sm')

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
#bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token = "AAAAAAAAAAAAAAAAAAAAAP1LRwEAAAAAn64BsczyRJiYHOVTTbN9415JH%2Fs%3DkzXwt5UPrYzTxL5sNVtqZD25ejB0UOtB4ayTcdZQfEUIwX26Qx"



In [2]:
def NEW_get_location(Xstring):
    
    doc = nlp_wk(Xstring)
#     displacy.render(doc, style="ent")
    locations = []
    locations.extend([ent.text for ent in doc.ents if ent.label_ in ['LOC']])
#     locations = [ent.text for ent in doc.ents if ent.label_ in ['LOC']]
    return locations

In [3]:
def get_userTimeline(userDetails): 
    tweet_array = []
    for user_id in userDetails:
        url = "https://api.twitter.com/2/users/{}/tweets".format(user_id)
        payload={}
        headers = {
          'Authorization': 'Bearer '+bearer_token
        }
        response = requests.request("GET", url, headers=headers, data=payload)

        for response_line in response.iter_lines():
            if response_line:
                json_response = json.loads(response_line)
                #Prints loction for user's timeline
                for tweet in json_response['data']:
                    locations = NEW_get_location(tweet['text'])
                    
#--------------------------> NEED TO WORK HERE   
                    if len(locations)>0 :
                        tweet_array.append([tweet['text'], user_id, locations[0]])
                
    return tweet_array

In [4]:
def get_stream():   
    query_melbRegions = ["melbourne"]
    
    with open('SA2Name_Melbourne.json') as f:
        melb_regions = json.load(f)
        for regions in melb_regions['features']:
            query_melbRegions.append(regions['properties']['sa2_name16'])

    n=10
    region_chunks = [query_melbRegions[i:i + n] for i in range(0, len(query_melbRegions), n)]
    tweet_array = []
    users_array = []

    for reg in region_chunks:
        
        reg_query = ' OR '.join(str(v) for v in reg)
        query = reg_query +" "+ "(beaches OR pubs OR stadiums OR Museu OR art Galleries OR Gardens OR Clubs OR Bars OR Restauraunt OR Hotels OR Motels OR Schools OR Universities OR Parlor OR Saloon OR Coffee Shop OR Casinos OR Hospitals)"

        tweetFields = "tweet.fields=created_at,lang"
        maxResults = "max_results=10"
        expansions = "expansions=author_id,geo.place_id"

        url = "https://api.twitter.com/2/tweets/search/recent?query="+query+"&"+tweetFields+"&"+maxResults+"&"+expansions+"&"

        payload={}
        headers = {
          'Authorization': 'Bearer '+bearer_token,
          'Cookie': 'guest_id=v1%3A162668876704133606; personalization_id="v1_QPzpEmtR+oVGPKUXrUpOIA=="'
        }

        response = requests.request("GET", url, headers=headers, data=payload)

        print(response.status_code)
        if response.status_code != 200:
            raise Exception(
                "Cannot get stream (HTTP {}): {}".format(
                    response.status_code, response.text
                )
            )    
        
        for response_line in response.iter_lines():
            if response_line:
                json_response = json.loads(response_line)                
                for users in json_response['includes']['users']:
                    users_array.append(users['id'])
                    
                for tweet in json_response['data']:

                    locations = NEW_get_location(tweet['text'])
    #--------------------------> NEED TO WORK HERE
                    if len(locations)>0 :
                        tweet_array.append([tweet['text'], tweet['author_id'], locations[0]])
    
    userTimelineData = get_userTimeline(users_array)
    UserTimelineDatFrame = pd.DataFrame(userTimelineData,columns=['tweet','author_id','locations'])
    UserTimelineDatFrame["address"] = UserTimelineDatFrame["locations"].apply(geocode)
    UserTimelineDatFrame["coordinates"] = UserTimelineDatFrame["address"].apply(lambda loc: tuple(loc.point) if loc else None)
    UserTimelineDatFrame[["latitude", "longitude", "altitude"]] = pd.DataFrame(UserTimelineDatFrame["coordinates"].tolist(), index=UserTimelineDatFrame.index)
    UserTimelineDatFrame.latitude.isnull().sum()
    UserTimelineDatFrame = UserTimelineDatFrame[pd.notnull(UserTimelineDatFrame["latitude"])]
    
    
#     For tweets 
    tweetDatFrame = pd.DataFrame(tweet_array,columns=['tweet','author_id','locations'])
    tweetDatFrame["address"] = tweetDatFrame["locations"].apply(geocode)
    
    tweetDatFrame["coordinates"] = tweetDatFrame["address"].apply(lambda loc: tuple(loc.point) if loc else None)
    tweetDatFrame[["latitude", "longitude", "altitude"]] = pd.DataFrame(tweetDatFrame["coordinates"].tolist(), index=tweetDatFrame.index)
    tweetDatFrame.latitude.isnull().sum()
    tweetDatFrame = tweetDatFrame[pd.notnull(tweetDatFrame["latitude"])]
    
    return tweetDatFrame,UserTimelineDatFrame

In [5]:
def main():
#     rules = get_rules()
#     delete = delete_all_rules(rules)
#     set = set_rules(delete)
    get_tweets,user_timeline = get_stream()  
    return get_tweets,user_timeline

In [7]:

folium_map = folium.Map(location=[59.338315,18.089960],zoom_start=2,tiles="CartoDB dark_matter")
   
    
if __name__ == "__main__":    
        final_tweets,final_user_timeline = main()
FastMarkerCluster(data=list(zip(final_tweets["latitude"].values, final_tweets["longitude"].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
# folium_map

FastMarkerCluster(data=list(zip(final_user_timeline["latitude"].values, final_user_timeline["longitude"].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


KeyboardInterrupt: 